Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Pendulum"

In [18]:
import numpy as np
from pendulum_env_extended import PendulumEnvExtended
import random 

In [19]:
env = PendulumEnvExtended(render_mode='rgb_array')

Discretización de los estados

In [20]:
x_space = np.linspace(-1, 1, 10)
y_space = np.linspace(-1, 1, 10)
vel_space = np.linspace(-8, 8, 100)
x_space

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])

Obtener el estado a partir de la observación

In [21]:
def get_state(obs):
    x, y, vel = obs
    x_bin = np.digitize(x, x_space)
    y_bin = np.digitize(y, y_space)
    vel_bin = np.digitize(vel, vel_space)
    return x_bin, y_bin, vel_bin

In [22]:
state = get_state(np.array([-0.4, 0.2, 0.3]))
state

(3, 6, 52)

Discretización de las acciones

In [23]:
actions = list(np.linspace(-2, 2, 10))
actions

[-2.0,
 -1.5555555555555556,
 -1.1111111111111112,
 -0.6666666666666667,
 -0.22222222222222232,
 0.22222222222222232,
 0.6666666666666665,
 1.1111111111111107,
 1.5555555555555554,
 2.0]

In [24]:
def get_sample_action():
    return random.choice(actions)

Inicilización de la tabla Q

In [25]:
Q = np.zeros((len(x_space) + 1, len(y_space) + 1, len(vel_space) + 1, len(actions)))
Q

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

Obtención de la acción a partir de la tabla Q

In [26]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action

Epsilon-Greedy Policy

In [27]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = get_sample_action()
        print('explore')
    # exploit
    else:
        action = optimal_policy(state, Q)
        print('exploit')
        
    return action

Ejemplo de episodio 

In [28]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
state = get_state(obs)
while not done:
    # Acción del modelo
    action = epsilon_greedy_policy(state, Q, 0.5)
    
    # Indice de la accion en Q
    action_idx = actions.index(action)
    
    # Acción del ambiente
    real_action = np.array([action])
     
    obs, reward, done, _, _ = env.step(real_action)
    next_state = get_state(obs)
    
   # Usar action_idx para actualizar Q
   
    total_reward += reward
    print('->', state, action, reward, obs, done)
    env.render()
print('total_reward', total_reward)

[-0.24699968  0.96901554  0.6567021 ]
explore
-> (4, 9, 54) 2.0 -3.3609058302594828 [-0.32759404  0.94481856  1.6834638 ] False
explore
-> (4, 9, 54) -2.0 -3.914724453869056 [-0.42445496  0.90544903  2.0920777 ] False
exploit
-> (4, 9, 54) -2.0 -4.478387457950209 [-0.53281033  0.8462347   2.4711645 ] False
explore
-> (4, 9, 54) 0.22222222222222232 -5.15918550619156 [-0.65853983  0.7525459   3.139174  ] False
exploit
-> (4, 9, 54) -2.0 -6.2320441430402065 [-0.7764772  0.6301454  3.4035833] False
exploit
-> (4, 9, 54) -2.0 -7.213311066525481 [-0.87617373  0.48199546  3.5761924 ] False
exploit
-> (4, 9, 54) -2.0 -8.245451639514942 [-0.94890577  0.3155596   3.6376889 ] False
exploit
-> (4, 9, 54) -2.0 -9.282759620265404 [-0.9898884   0.14184836  3.5743587 ] False
explore
-> (4, 9, 54) 2.0 -10.277190014771575 [-0.99839246 -0.05667846  3.9807448 ] False
explore
-> (4, 9, 54) 1.5555555555555554 -11.103560862586884 [-0.9650172  -0.26218653  4.1715693 ] False
explore
-> (4, 9, 54) -2.0 -10.0173